In [35]:
import geopandas as gpd

# Load the polygon GeoJSON
polygon_gdf = gpd.read_file("polygon.geojson")

# Load the county boundaries (replace 'counties.geojson' with your actual data file)
counties_gdf = gpd.read_file("counties2.geojson")

# Ensure the CRS (Coordinate Reference System) matches
polygon_gdf = polygon_gdf.to_crs(counties_gdf.crs)

# Calculate the total area of the farm (polygon)
farm_area = polygon_gdf.geometry.area.sum()

# Perform spatial join to find counties that intersect with the polygon
intersecting_counties = counties_gdf[counties_gdf.geometry.intersects(polygon_gdf.unary_union)]

# Calculate the intersection area for each county
intersecting_counties["intersection_area"] = intersecting_counties.geometry.intersection(
    polygon_gdf.unary_union
).area

# Calculate the percentage of the farm area covered by each county
intersecting_counties["percentage_of_farm"] = (
    intersecting_counties["intersection_area"] / farm_area * 100
)

# Display results
intersecting_counties

/var/folders/f3/rbsn08md2csgshqkwfk85w040000gn/T/ipykernel_28263/2973285160.py:13: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  farm_area = polygon_gdf.geometry.area.sum()
/var/folders/f3/rbsn08md2csgshqkwfk85w040000gn/T/ipykernel_28263/2973285160.py:16: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  intersecting_counties = counties_gdf[counties_gdf.geometry.intersects(polygon_gdf.unary_union)]
/var/folders/f3/rbsn08md2csgshqkwfk85w040000gn/T/ipykernel_28263/2973285160.py:20: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  polygon_gdf.unary_union
/var/folders/f3/rbsn08md2csgshqkwfk85w040000gn/T/ipykernel_28263/2973285160.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to

,OBJECTID,DNR_REGION_NAME,DNR_CNTY_CODE,COUNTY_NAME,COUNTY_FIPS_CODE,SHAPEAREA,SHAPELEN,geometry,intersection_area,percentage_of_farm
11,972,South Central Region,13,Dane,25,3.204610e+09,231224.530044,"POLYGON ((-89.36309 43.28132, -89.36312 43.294...",0.000347,29.552033
16,977,South Central Region,14,Dodge,27,2.348412e+09,195437.668076,"POLYGON ((-88.40097 43.61644, -88.40114 43.630...",0.000511,43.576725
18,979,South Central Region,11,Columbia,21,2.060374e+09,222534.705663,"POLYGON ((-89.12693 43.64276, -89.12692 43.643...",0.000315,26.871242


In [37]:
def get_countyinfo(geojson):
    polygon_gdf = gpd.read_file(geojson)
    counties_gdf = gpd.read_file("counties2.geojson")
    polygon_gdf = polygon_gdf.to_crs(counties_gdf.crs)
    farm_area = polygon_gdf.geometry.area.sum()
    intersecting_counties = counties_gdf[counties_gdf.geometry.intersects(polygon_gdf.unary_union)]
    intersecting_counties["intersection_area"] = intersecting_counties.geometry.intersection(
        polygon_gdf.unary_union
    ).area
    intersecting_counties["percentage_of_farm"] = (
        intersecting_counties["intersection_area"] / farm_area * 100
    )
    return intersecting_counties[["COUNTY_NAME", "percentage_of_farm"]].to_dict(orient="records")

In [ ]:
# rename county column from COUNTY_NAME to county
intersecting_counties.rename(columns={"COUNTY_NAME": "county"}, inplace=True)
intersecting_counties[["COUNTY_NAME", "percentage_of_farm"]].to_dict(orient="records")

[{'COUNTY_NAME': 'Dane', 'percentage_of_farm': 29.5520332347441},
 {'COUNTY_NAME': 'Dodge', 'percentage_of_farm': 43.57672461782967},
 {'COUNTY_NAME': 'Columbia', 'percentage_of_farm': 26.871242147426543}]